# Image Embeddings with CLIP

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/etl/images_clip_to_qdrant.ipynb)

Generate image embeddings using OpenAI CLIP and store them in Qdrant for visual similarity search.

In [ ]:
!pip install -q qdrant-client transformers pillow torch

In [ ]:
import numpy as np
from PIL import Image
from transformers import CLIPModel, CLIPProcessor
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

In [ ]:
client = QdrantClient(":memory:")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
client.create_collection(
    collection_name="images",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

In [ ]:
# Generate synthetic images for demo (colored squares)
colors = {
    "red": (255, 0, 0),
    "green": (0, 255, 0),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0),
    "purple": (128, 0, 128),
}

points = []
for idx, (name, rgb) in enumerate(colors.items()):
    img = Image.new("RGB", (224, 224), rgb)
    inputs = processor(images=img, return_tensors="pt")
    embedding = model.get_image_features(**inputs).detach().numpy()[0].tolist()

    points.append(
        PointStruct(
            id=idx,
            vector=embedding,
            payload={"color": name, "rgb": list(rgb)},
        )
    )

client.upsert(collection_name="images", points=points)
print(f"Indexed {len(points)} images")

In [ ]:
# Search with text query (CLIP cross-modal search)
text_inputs = processor(text=["a warm sunset color"], return_tensors="pt")
text_embedding = model.get_text_features(**text_inputs).detach().numpy()[0].tolist()

results = client.search(
    collection_name="images",
    query_vector=text_embedding,
    limit=3,
)

print("Query: 'a warm sunset color'")
for r in results:
    print(f"  Score: {r.score:.4f} | {r.payload['color']}")